## Change Data Capture
### with PySpark

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col

In [2]:
spark = SparkSession.builder.appName("CDC").getOrCreate()

your 131072x1 screen size is bogus. expect trouble


24/04/04 22:39:30 WARN Utils: Your hostname, DESKTOP-SLEQT56 resolves to a loopback address: 127.0.1.1; using 172.25.13.138 instead (on interface eth0)
24/04/04 22:39:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/04/04 22:39:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


1. Cargamos el archivo __FULL LOAD__ y le cambiamos el nombre a las variables

In [3]:
fldf = spark.read.csv("/mnt/d/Proyectos/Tutorial-SparkAWS/CDC_data/LOAD00000001.csv")
fldf = fldf.withColumnRenamed("_c0", "id").\
    withColumnRenamed("_c1", "fullName").\
        withColumnRenamed("_c2", "City")
fldf.show(5)

+---+----------------+-------------+
| id|        fullName|         City|
+---+----------------+-------------+
|  0|Herman Zimmerman|Oklahoma City|
|  1|        Lisa Ray|     Columbus|
|  2|  Terrell Reeves| Jacksonville|
|  3|   Steve Goodwin|    Charlotte|
|  4|       Leah Tran|      Detroit|
+---+----------------+-------------+
only showing top 5 rows



2. Creamos el archivo el un nuevo directorio

In [4]:
fldf.write.mode("overwrite").\
    csv("/mnt/d/Proyectos/Tutorial-SparkAWS/CDC_data/final_output/finalFile.csv")

3. Cargamos el archivo __Replication onGoing__ que contiene los cambios

In [5]:
udf = spark.read.csv("/mnt/d/Proyectos/Tutorial-SparkAWS/CDC_data/20240405-003052555.csv")
udf = udf.withColumnRenamed("_c0", "action").\
    withColumnRenamed("_c1", "id").\
        withColumnRenamed("_c2", "fullName").\
            withColumnRenamed("_c3", "city")
            

3.1 Cargamos el archivo __fullLoad__ que copiamos en el paso .2 

In [6]:
ffdf = spark.read.csv("/mnt/d/Proyectos/Tutorial-SparkAWS/CDC_data/final_output/finalFile.csv")
ffdf = fldf.withColumnRenamed("_c0", "id").\
        withColumnRenamed("_c1", "fullName").\
            withColumnRenamed("_c2", "City")
ffdf.show(5)

+---+----------------+-------------+
| id|        fullName|         City|
+---+----------------+-------------+
|  0|Herman Zimmerman|Oklahoma City|
|  1|        Lisa Ray|     Columbus|
|  2|  Terrell Reeves| Jacksonville|
|  3|   Steve Goodwin|    Charlotte|
|  4|       Leah Tran|      Detroit|
+---+----------------+-------------+
only showing top 5 rows



4. Procesamos el archivo __Replication onGoing__ por cada tipo de Operacion

In [7]:
for row in udf.collect():
    print("{} - {}".format(row, type(row)))
    if row['action'] == 'U':
        print('Update')
    elif row['action'] == 'D':
        print('Delete')
    else:
        print('Insert')

Row(action='U', id='7', fullName='ABC XYZ', city='Phoenix') - <class 'pyspark.sql.types.Row'>
Update
Row(action='I', id='130', fullName='Alica Bing', city='New York') - <class 'pyspark.sql.types.Row'>
Insert
Row(action='I', id='131', fullName='Malinda Bing', city='Detroit') - <class 'pyspark.sql.types.Row'>
Insert
Row(action='I', id='132', fullName='Chandler Bing', city='Portland') - <class 'pyspark.sql.types.Row'>
Insert
Row(action='U', id='8', fullName='ABC XYZ', city='Denver') - <class 'pyspark.sql.types.Row'>
Update
Row(action='D', id='10', fullName='Jack Hicks', city='Houston') - <class 'pyspark.sql.types.Row'>
Delete
